In [17]:
import pandas as pd 
import numpy as np
import time
import math
import random
import os
from collections import Counter
import copy
import pickle
import os 
import csv
import re
import Levenshtein

In [18]:
# namedataset="Helpdesk"
# namedataset="BPI_Challenge_2012_W_Complete"
namedataset="BPI_Challenge_2012_W"
# namedataset="BPI_Challenge_2012_A"
# namedataset="BPI_Challenge_2012_O"

In [19]:
for step in range(5):
    # train_val_file="processed_datasets/"+namedataset+"/fold"+str(step)+"/train_val_"+namedataset+".csv"
    train_file="processed_datasets/"+namedataset+"/fold"+str(step)+"/train_"+namedataset+".csv"
    val_file="processed_datasets/"+namedataset+"/fold"+str(step)+"/val_"+namedataset+".csv"
    test_file="processed_datasets/"+namedataset+"/fold"+str(step)+"/test_"+namedataset+".csv"
    df_test=pd.read_csv(test_file)
    alpha=0.3
    fold="fold"+str(step)
    pattern_act_dict=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/pattern_act_dict_"+str(alpha)+".pkl","rb"))
    pattern_act_time_dict=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/pattern_act_time_dict_"+str(alpha)+".pkl","rb"))
    pattern_act_res_dict=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/pattern_act_res_dict_"+str(alpha)+".pkl","rb"))
    res_cluster_label=pickle.load(open("datasets_results/"+namedataset+"/"+fold+"/res_cluster_label_"+str(alpha)+".pkl","rb"))
    res_cluster_num=max(res_cluster_label)+1

    pattern_act_sup_list=[]
    for key in pattern_act_dict:
        # print(len(pattern_act_dict[key]))
        pattern_act_sup_list.append(len(pattern_act_dict[key]))

    pattern_act_res_sup_list=[]
    for key in pattern_act_res_dict:
        # print(len(pattern_act_dict[key]))
        pattern_act_res_sup_list.append(len(pattern_act_res_dict[key]))

    pattern_act_time_sup_list=[]
    for key in pattern_act_time_dict:
        # print(len(pattern_act_dict[key]))
        pattern_act_time_sup_list.append(len(pattern_act_time_dict[key]))

    try:
        df_test.columns = ["CaseID", "Activity", "Resource", "Timestamp","amount"]
    except:
        df_test.columns = ["CaseID", "Activity", "Resource", "Timestamp"]

    df_train=pd.read_csv(train_file)
    try:
        df_train.columns = ["CaseID", "Activity", "Resource", "Timestamp","amount"]
    except:
        df_train.columns = ["CaseID", "Activity", "Resource", "Timestamp"]

    df_valid=pd.read_csv(val_file)
    try:
        df_valid.columns = ["CaseID", "Activity", "Resource", "Timestamp","amount"]
    except:
        df_valid.columns = ["CaseID", "Activity", "Resource", "Timestamp"]

    df_train.fillna(1, inplace=True)
    df_test.fillna(1,inplace=True)
    df_valid.fillna(1,inplace=True)

    cont_trace = df_train['CaseID'].value_counts(dropna=False)
    cont_test_trace = df_test['CaseID'].value_counts(dropna=False)
    cont_val_trace = df_valid['CaseID'].value_counts(dropna=False)

    max_trace = max(cont_trace)
    max_test_trace = max(cont_test_trace)
    max_val_trace = max(cont_val_trace)
    real_max=max(max_trace,max_test_trace,max_val_trace)
    df_full=df_train.append(df_test).append(df_valid)
    mean_trace=int(round(np.mean(df_full['CaseID'].value_counts(dropna=False))))
    
    test_listOfResource=df_test["Resource"].unique().tolist()
    train_listOfResource=df_train["Resource"].unique().tolist()
    valid_listOfResource=df_valid["Resource"].unique().tolist()

    test_resourceset=set(test_listOfResource)
    train_resourceset=set(train_listOfResource)
    valid_resourceset=set(valid_listOfResource)
    all_resource=list(valid_resourceset.union(test_resourceset.union(train_resourceset)))
    all_resource.sort()
    listOfresourcesInt = list(range(0, len(all_resource)))
    resourceMapping=dict(zip(all_resource,listOfresourcesInt))

    test_listOfEvents=df_test["Activity"].unique().tolist()
    train_listOfEvents=df_train["Activity"].unique().tolist()
    valid_listOfEvents=df_valid["Activity"].unique().tolist()
    test_eventset=set(test_listOfEvents)
    train_eventset=set(train_listOfEvents)
    valid_eventset=set(valid_listOfEvents)
    all_events=list(valid_eventset.union(test_eventset.union(train_eventset)))
    all_events.sort()
    listOfeventsInt = list(range(0, len(all_events)))
    mapping = dict(zip(all_events, listOfeventsInt))
    df_test.Activity = [mapping[item] for item in df_test.Activity]
    # 将活动名称和id对应
    df_train.Activity = [mapping[item] for item in df_train.Activity]
    # 将活动名称和id对应
    df_valid.Activity = [mapping[item] for item in df_valid.Activity]

    df_test.Resource = [resourceMapping[item] for item in df_test.Resource]
    df_train.Resource = [resourceMapping[item] for item in df_train.Resource]
    df_valid.Resource = [resourceMapping[item] for item in df_valid.Resource]

    act = df_train.groupby('CaseID', sort=False).agg({'Activity': lambda x: list(x)})
    res = df_train.groupby('CaseID', sort=False).agg({'Resource': lambda x: list(x)})
    temp = df_train.groupby('CaseID', sort=False).agg({'Timestamp': lambda x: list(x)})

    act_valid = df_valid.groupby('CaseID', sort=False).agg({'Activity': lambda x: list(x)})
    res_valid = df_valid.groupby('CaseID', sort=False).agg({'Resource': lambda x: list(x)})
    temp_valid = df_valid.groupby('CaseID', sort=False).agg({'Timestamp': lambda x: list(x)})

    act_test = df_test.groupby('CaseID', sort=False).agg({'Activity': lambda x: list(x)})
    res_test = df_test.groupby('CaseID', sort=False).agg({'Resource': lambda x: list(x)})
    temp_test = df_test.groupby('CaseID', sort=False).agg({'Timestamp': lambda x: list(x)})

    act,res,temp=act.loc[:,"Activity"].values,res.loc[:,"Resource"].values,temp.loc[:,"Timestamp"].values
    act_valid,res_valid,temp_valid=act_valid.loc[:,"Activity"].values,res_valid.loc[:,"Resource"].values,temp_valid.loc[:,"Timestamp"].values
    act_test,res_test,temp_test=act_test.loc[:,"Activity"].values,res_test.loc[:,"Resource"].values,temp_test.loc[:,"Timestamp"].values

    train_res_cluster_log=[[res_cluster_label[resource] for resource in res_trace] for res_trace in res]
    valid_res_cluster_log=[[res_cluster_label[resource] for resource in res_trace] for res_trace in res_valid]
    test_res_cluster_log=[[res_cluster_label[resource] for resource in res_trace] for res_trace in res_test]

    
    for act_trace in act:
        act_trace.append(len(all_events))
    for res_trace in res:
        res_trace.append(len(all_resource))
    for time_trace in temp:
        time_trace.append(time_trace[-1])

    for act_trace in act_valid:
        act_trace.append(len(all_events))
    for res_trace in res_valid:
        res_trace.append(len(all_resource))
    for time_trace in temp_valid:
        time_trace.append(time_trace[-1])

    for act_trace in act_test:
        act_trace.append(len(all_events))
    for res_trace in res_test:
        res_trace.append(len(all_resource))
    for time_trace in temp_test:
        time_trace.append(time_trace[-1])
    
    def distance(prefix,pattern,mode=1):
        distance=0
        if mode :
            a=1 
        else:
            # b=2
            distance=Levenshtein.distance(prefix,pattern)
        return distance

    def prefix_match(prefix,pattern_dict,sup_list,alpha):
        distances=[]
        patterns=[]
        prefix=[str(key) for key in prefix]
        # prefix=prefix.split(",")
        for key,values in pattern_dict.items():
            key=key.split(",")
            distances.append(Levenshtein.distance(key,prefix))
            patterns.append(key)
        
        min_dis,max_dis=min(distances),max(distances)
        if min_dis==max_dis:
            distances=[0]*len(distances)
        else:
            for i in range(len(distances)):
                try:
                    distances[i]=1-(distances[i]-min_dis)/(max_dis-min_dis)
                except ZeroDivisionError :
                    print("what???")             
                # distances[i]=1-(distances[i]-min_dis)/(max_dis-min_dis)
                #             
        matchs=[alpha*dis+(1-alpha)*sup for dis,sup in zip(distances,sup_list)]
        matched_pattern=patterns[matchs.index(max(matchs))]
        return matched_pattern
    
    rg=re.compile("\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}")
    def time_format(ttime):
        ttime=rg.search(ttime).group(0)
        try:
            date_format_str = '%Y/%m/%d %H:%M:%S.%f'
            conversion = time.mktime(time.strptime(ttime, date_format_str))
        except:
            date_format_str = '%Y-%m-%d %H:%M:%S'
            conversion = time.mktime(time.strptime(ttime, date_format_str))
        return conversion

    maxDay=pickle.load(open("datasets_results/"+namedataset+"/"+"fold"+str(step)+"/maxInterval_"+str(alpha)[:3]+".pkl","rb"))

    def get_max_pattern_length(pattern):
        pattern_key_list=[key.split(",") for key in  pattern]
        pattern_key_length=[len(key) for key in  pattern_key_list]
        pattern_key_length.sort()
        return pattern_key_length[-1]

    act_pattern_length=get_max_pattern_length(pattern_act_dict)
    act_res_pattern_length=get_max_pattern_length(pattern_act_res_dict)
    act_time_pattern_length=get_max_pattern_length(pattern_act_time_dict)

    def prefix_gen(act,res,res_cluster_log,temp):
        day_seconds=60*60*24*3
        max_day=maxDay
        max_last,max_difference=0,0
        min_last=1e6
        min_remain,max_remain=1e6,0
        data=[]
        act_pattern_list=[]
        act_res_pattern_list=[]
        act_time_pattern_list=[]
        label=[]
        for act_trace,res_trace,res_cluster_trace,time_trace in zip(act,res,res_cluster_log,temp):
            for i in range(1,len(act_trace)):
                prefix_act=act_trace[:i]
                prefix_res=res_trace[:i]
                prefix_res_cluster=res_cluster_trace[:i]
                # 获取时间向量
                temp_time=[time_format(ttime) for ttime in time_trace[:i]]
                # 前缀的持续时间轨迹
                prefix_last=[time-temp_time[0] for time in temp_time]
                # 前缀的间隔时间轨迹
                prefix_difference=[ temp_time[m]-temp_time[m-1] if m-1>=0 else 0 for m,time in enumerate (temp_time)]
                max_last=max(max_last,max(prefix_last))
                min_last=min(min_last,min(prefix_last))
                max_difference=max(max_difference,max(prefix_difference))
                prefix_difference_interval=[math.floor(different/day_seconds) if (different/day_seconds)<maxDay else maxDay  for different in prefix_difference]
                act_time_prefix=[str(activity)+";"+str(difference) for activity,difference in zip(prefix_act,prefix_difference_interval)]
                act_res_prefix=[str(activity)+";"+str(resource) for activity,resource in zip(prefix_act,prefix_res_cluster)]
                # pattern=prefix_match(prefix_act,pattern_act_dict,0.5)
                act_pattern=prefix_match(prefix_act,pattern_act_dict,pattern_act_sup_list,0.7)
                act_res_pattern=prefix_match(act_res_prefix,pattern_act_res_dict,pattern_act_res_sup_list,0.7)
                act_time_pattern=prefix_match(act_time_prefix,pattern_act_time_dict,pattern_act_time_sup_list,0.7)

                act_label=act_trace[i]
                res_label=res_trace[i]
                # 获取真实的下一时间间隔值
                real_time=(time_format(time_trace[i]))-(time_format(time_trace[i-1])) if i-1>=0 else 0
                time_label=(real_time//day_seconds) if (real_time//day_seconds)<maxDay else maxDay 
                # 获取真实的剩余时间值
                real_remain=(time_format(time_trace[-1]))-(time_format(time_trace[i]))
                remain_label=(real_remain//day_seconds) if (real_remain//day_seconds)<maxDay else maxDay
                min_remain=min(min_remain,real_remain)
                max_remain=max(max_remain,real_remain)
                # 处理前缀数据，将其转为mean_trace长度
                if i>=mean_trace:
                    prefix_act=prefix_act[-mean_trace:]
                    prefix_res=prefix_res[-mean_trace:]
                    prefix_last=prefix_last[-mean_trace:]
                    prefix_difference=prefix_difference[-mean_trace:]
                else:
                    prefix_act=[len(all_events)+1 for j in range(mean_trace-i)]+prefix_act
                    prefix_res=[len(all_resource)+1 for j in range(mean_trace-i)]+prefix_res
                    prefix_last=[0 for j in range(mean_trace-i)]+prefix_last
                    prefix_difference=[0 for j in range(mean_trace-i)]+prefix_difference
                

                # 将数据加入列表中
                # 组合data
                prefix_act=np.array(prefix_act).reshape(mean_trace,1)
                prefix_res=np.array(prefix_res).reshape(mean_trace,1)
                prefix_last=np.array(prefix_last).reshape(mean_trace,1)
                prefix_difference=np.array(prefix_difference).reshape(mean_trace,1)
                tdata=np.concatenate((prefix_act,prefix_res,prefix_last,prefix_difference),axis=1) #tdata.shape [mean_trae,4]
                data.append(tdata)

                act_pattern=[int(key) for key in act_pattern]
                act_res_pattern=[[int(key.split(";")[0]),int(key.split(";")[1])] for key in act_res_pattern]
                act_time_pattern=[[int(key.split(";")[0]),int(key.split(";")[1])] for key in act_time_pattern]

                if len(act_pattern)<act_pattern_length:
                    act_pattern=[len(all_events)+1 for j in range(act_pattern_length-len(act_pattern))] + act_pattern
                if len(act_res_pattern)<act_res_pattern_length:
                    act_res_pattern=[[len(all_events)+1,res_cluster_num] for j in range(act_res_pattern_length-len(act_res_pattern))] + act_res_pattern
                if len(act_time_pattern)<act_time_pattern_length:
                    act_time_pattern=[[len(all_events)+1,maxDay+1] for j in range(act_time_pattern_length-len(act_time_pattern))] + act_time_pattern
                
                act_pattern_list.append(act_pattern)
                act_res_pattern_list.append(act_res_pattern)
                act_time_pattern_list.append(act_time_pattern)
                # 组合label
                tlabel=np.array([act_label,res_label,time_label,remain_label,real_time,real_remain])
                label.append(tlabel)
        
        return data,label,act_pattern_list,act_res_pattern_list,act_time_pattern_list,max_last,max_difference,min_last,min_remain,max_remain

    train_data,train_label,train_act_pattern_list,train_act_res_pattern_list,train_act_time_pattern_list,train_max_last,train_max_dif,train_min_last,train_min_remain,train_max_remain=prefix_gen(act,res,train_res_cluster_log,temp)
    valid_data,valid_label,valid_act_pattern_list,valid_act_res_pattern_list,valid_act_time_pattern_list,valid_max_last,valid_max_dif,valid_min_last,valid_min_remain,valid_max_remain=prefix_gen(act_valid,res_valid,valid_res_cluster_log,temp_valid)
    test_data,test_label,test_act_pattern_list,test_act_res_pattern_list,test_act_time_pattern_list,test_max_last,test_max_dif,test_min_last,test_min_remain,test_max_remain=prefix_gen(act_test,res_test,test_res_cluster_log,temp_test)

    # if not os.path.exists("datasets_results/"+namedataset+"/"+fold+"/")
    pickle.dump([train_max_remain,valid_max_remain,test_max_remain],open("datasets_results/"+namedataset+"/"+fold+"/time_normlize.pkl","wb"))
    real_max_dif=max([train_max_dif,valid_max_dif,test_max_dif])
    real_max_remain=max([train_max_remain,valid_max_remain,test_max_remain])
    real_max_last=max([train_max_last,valid_max_last,test_max_last])
    for data in train_data:
        for row in data:
            row[2]=(row[2])/(real_max_last)
            row[3]=row[3]/real_max_dif

    for data in train_label:
        data[4]=data[4]/real_max_dif
        # data[5]=(data[5]-min_remain)/(max_remain-min_remain)
        data[5]=(data[5])/(real_max_remain)

    for data in valid_data:
        for row in data:
            row[2]=(row[2])/(real_max_last)
            row[3]=row[3]/real_max_dif

    for data in valid_label:
        data[4]=data[4]/real_max_dif
        # data[5]=(data[5]-min_remain)/(max_remain-min_remain)
        data[5]=(data[5])/(real_max_remain)
    
    for data in test_data:
        for row in data:
            row[2]=(row[2])/(real_max_last)
            row[3]=row[3]/real_max_dif

    for data in test_label:
        data[4]=data[4]/real_max_dif
        # data[5]=(data[5]-min_remain)/(max_remain-min_remain)
        data[5]=(data[5])/(real_max_remain)

    if not os.path.exists("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)):
        os.mkdir("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace))
    
    pickle.dump(train_data,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"train_data_"+str(alpha)+".pkl","wb"))
    pickle.dump(train_label,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"train_label_"+str(alpha)+".pkl","wb"))
    pickle.dump(train_act_pattern_list,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"train_act_pattern_list_"+str(alpha)+".pkl","wb"))
    pickle.dump(train_act_res_pattern_list,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"train_act_res_pattern_list_"+str(alpha)+".pkl","wb"))
    pickle.dump(train_act_time_pattern_list,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"train_act_time_pattern_list_"+str(alpha)+".pkl","wb"))

    pickle.dump(test_data,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"test_data_"+str(alpha)+".pkl","wb"))
    pickle.dump(test_label,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"test_label_"+str(alpha)+".pkl","wb"))
    pickle.dump(test_act_pattern_list,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"test_act_pattern_list_"+str(alpha)+".pkl","wb"))
    pickle.dump(test_act_res_pattern_list,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"test_act_res_pattern_list_"+str(alpha)+".pkl","wb"))
    pickle.dump(test_act_time_pattern_list,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"test_act_time_pattern_list_"+str(alpha)+".pkl","wb"))

    pickle.dump(valid_data,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"valid_data_"+str(alpha)+".pkl","wb"))
    pickle.dump(valid_label,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"valid_label_"+str(alpha)+".pkl","wb"))
    pickle.dump(valid_act_pattern_list,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"valid_act_pattern_list_"+str(alpha)+".pkl","wb"))
    pickle.dump(valid_act_res_pattern_list,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"valid_act_res_pattern_list_"+str(alpha)+".pkl","wb"))
    pickle.dump(valid_act_time_pattern_list,open("datasets_results/"+namedataset+"/"+fold+"/"+"trace_length_"+str(mean_trace)+"/"+"valid_act_time_pattern_list_"+str(alpha)+".pkl","wb"))




/tmp/ipykernel_26426/1779962214.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_full=df_train.append(df_test).append(df_valid)
/tmp/ipykernel_26426/1779962214.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_full=df_train.append(df_test).append(df_valid)


OSError: [Errno 28] No space left on device: 'datasets_results/BPI_Challenge_2012_W/fold1/trace_length_18'